In [1]:
%load_ext tensorboard
# Clear any logs from previous runs
!rm -rf ./logs/

In [2]:
import numpy as np
import pandas as pd
import time, os

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.utils import to_categorical
from tensorboard.plugins.hparams import api as hp

from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.models import Model

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split as split

import random
from math import floor
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from scipy.integrate import odeint
import scipy
import pandas as pd

In [3]:
df=pd.read_csv('data/jla.csv')
N = len(df.values)
randomize = np.random.permutation(N)
data = df.values[randomize]
N = len(df.values)
z = data[:,0] 
y = data[:,1:3] ### coge el resto de variables a predecir 
y[:,1] = y[:,1]**2+data[:,2]
np.shape(y)

(740, 2)

In [4]:
scalerz = StandardScaler()
scalerz.fit(z.reshape(-1,1))
z = scalerz.transform(z.reshape(-1,1))

In [5]:
### Modificar para incluir phanteon como test
split = 0.75
ntrain = int(split * len(z))
indx = [ntrain]
X_train, X_test = np.split(z, indx)
Y_train, Y_test = np.split(y, indx)
np.shape(X_train), np.shape(X_test), Y_train.shape, Y_test.shape

((555, 1), (185, 1), (555, 2), (185, 2))

In [6]:
# loss -> val_loss
callbacks = [tf.keras.callbacks.EarlyStopping(monitor='loss', mode='min',
                                   min_delta=0.01,
                                   patience=50,
                                   restore_best_weights=True)
                                   ]

n_cols = 1

In [7]:
HP_BATCHSIZE = hp.HParam('batch_size', hp.Discrete([1, 4, 8]))
HP_LAYERS =    hp.HParam('layers', hp.Discrete([2, 3, 4]))
HP_NUM_UNITS = hp.HParam('num_units', hp.Discrete([50, 100, 150, 200]))
HP_LEARNING  = hp.HParam('learning_rate', hp.Discrete([1e-5,1e-4,1e-3]))
# HP_NUM_UNITS3 = hp.HParam('num_units3', hp.Discrete([50, 100, 150, 200]))
# HP_NUM_UNITS4 = hp.HParam('num_units4', hp.Discrete([2, 5, 10]))
# HP_DROPOUT = hp.HParam('dropout', hp.RealInterval(0.0, 0.2))
# HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'sgd']))
# HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'Adadelta']))
# HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam']))

In [8]:
epochs=500

In [9]:
# sess = tf.Session()
# writer = tf.summary.FileWriter("/tmp/tfvgg", sess.graph)
# init = tf.initialize_all_variables()
# sess.run(init)
# with tf.summary.create_file_writer('logs/hparam_tuning').as_default():
#     hp.hparams_config(
#         hparams=[HP_NUM_UNITS1, HP_NUM_UNITS2, HP_NUM_UNITS3, HP_NUM_UNITS4,
#                  HP_OPTIMIZER, HP_BATCHSIZE],
#         metrics=[hp.Metric('loss', display_name="Loss")])

In [10]:
# METRIC_ACCURACY = 'accuracy'
with tf.summary.create_file_writer('logs/hparam_tuning3').as_default():
# with tf.summary.FileWriter('logs/hparam_tuning', sess.graph):
#     init = tf.initialize_all_variables()
#     sess.run(init)
    hp.hparams_config(
        hparams=[HP_LAYERS,
                 HP_NUM_UNITS,
                 HP_LEARNING, 
                 HP_BATCHSIZE],
        metrics=[hp.Metric('loss', display_name="Loss")])

In [11]:
def train_test_model(hparams):    
    
    # Train LSTM model and predict on validation set
    model = tf.keras.Sequential()
    model.add(Input(shape=(int(X_train.shape[1]),)))
    model.add(Dense(hparams[HP_NUM_UNITS], input_shape=(int(X_train.shape[1]),)))
    
    for i in range(hparams[HP_LAYERS]):        
        model.add(Dense(hparams[HP_NUM_UNITS], activation='relu'))
    model.add(Dense(2, activation='linear'))
     
    optimizer = tf.keras.optimizers.Adam(learning_rate=hparams[HP_LEARNING], beta_1=0.9, beta_2=0.999, epsilon=1e-3)
    model.compile(
            optimizer=optimizer,
            loss='mse', 
            metrics=['mean_squared_error'])
    
    # Run with 1 epoch to speed things up for demo purposes

    model.fit(X_train, Y_train, epochs=epochs, validation_data=(X_test, Y_test),
              callbacks=callbacks, batch_size=hparams[HP_BATCHSIZE], shuffle=True, verbose=0)

    _, loss = model.evaluate(X_test, Y_test)
    
    return loss

In [12]:
def run(run_dir, hparams):
    with tf.summary.create_file_writer(run_dir).as_default():
        hp.hparams(hparams)  # record the values used in this trial
        loss = train_test_model(hparams)
        tf.summary.scalar("loss", loss, step=1)
        return loss

In [13]:
session_num = 0
datos = []

for deep_layers in HP_LAYERS.domain.values:
    for num_units in HP_NUM_UNITS.domain.values:
        for learning_rate in HP_LEARNING.domain.values:
            for batch_size in HP_BATCHSIZE.domain.values:
                t = time.time()
                hparams = {

                    HP_LAYERS: deep_layers,
                    HP_NUM_UNITS: num_units,
                    HP_LEARNING: learning_rate,
                    HP_BATCHSIZE: batch_size,
                }
                run_name = "run-%d" % session_num
                print('\n--- Starting trial: %s' % run_name)
                print({h.name: hparams[h] for h in hparams})
                score = run('logs/hparam_tuning2/' + run_name, hparams)
                t = time.time()-t
                session_num += 1
                print("Loss:", score, "Tiempo transcurrido:", t)
            
            datos.append([deep_layers, num_units, learning_rate, batch_size, score, t])

print(session_num)


--- Starting trial: run-0
{'layers': 2, 'num_units': 50, 'learning_rate': 1e-05, 'batch_size': 1}
6/6 [==============================] - 0s 2ms/step - loss: 0.0639 - mean_squared_error: 0.0639
Loss: 0.06393277645111084 Tiempo transcurrido: 289.9850859642029

--- Starting trial: run-1
{'layers': 2, 'num_units': 50, 'learning_rate': 1e-05, 'batch_size': 4}
6/6 [==============================] - 0s 1ms/step - loss: 0.0720 - mean_squared_error: 0.0720
Loss: 0.07195807248353958 Tiempo transcurrido: 128.4882047176361

--- Starting trial: run-2
{'layers': 2, 'num_units': 50, 'learning_rate': 1e-05, 'batch_size': 8}
6/6 [==============================] - 0s 1ms/step - loss: 0.1027 - mean_squared_error: 0.1027
Loss: 0.1027413010597229 Tiempo transcurrido: 68.45728731155396

--- Starting trial: run-3
{'layers': 2, 'num_units': 50, 'learning_rate': 0.0001, 'batch_size': 1}
6/6 [==============================] - 0s 2ms/step - loss: 0.0420 - mean_squared_error: 0.0420
Loss: 0.04195890203118324 Tie

6/6 [==============================] - 0s 2ms/step - loss: 0.0633 - mean_squared_error: 0.0633
Loss: 0.0633159726858139 Tiempo transcurrido: 81.47552371025085

--- Starting trial: run-20
{'layers': 2, 'num_units': 150, 'learning_rate': 1e-05, 'batch_size': 8}
6/6 [==============================] - 0s 2ms/step - loss: 0.0713 - mean_squared_error: 0.0713
Loss: 0.07127493619918823 Tiempo transcurrido: 61.50955939292908

--- Starting trial: run-21
{'layers': 2, 'num_units': 150, 'learning_rate': 0.0001, 'batch_size': 1}
6/6 [==============================] - 0s 2ms/step - loss: 0.0580 - mean_squared_error: 0.0580
Loss: 0.057983506470918655 Tiempo transcurrido: 80.66062140464783

--- Starting trial: run-22
{'layers': 2, 'num_units': 150, 'learning_rate': 0.0001, 'batch_size': 4}
6/6 [==============================] - 0s 2ms/step - loss: 0.0398 - mean_squared_error: 0.0398
Loss: 0.039806388318538666 Tiempo transcurrido: 38.83181285858154

--- Starting trial: run-23
{'layers': 2, 'num_units':

6/6 [==============================] - 0s 2ms/step - loss: 0.0743 - mean_squared_error: 0.0743
Loss: 0.07429643720388412 Tiempo transcurrido: 143.05382204055786

--- Starting trial: run-40
{'layers': 3, 'num_units': 50, 'learning_rate': 0.0001, 'batch_size': 4}
6/6 [==============================] - 0s 2ms/step - loss: 0.0434 - mean_squared_error: 0.0434
Loss: 0.04344772547483444 Tiempo transcurrido: 41.74489641189575

--- Starting trial: run-41
{'layers': 3, 'num_units': 50, 'learning_rate': 0.0001, 'batch_size': 8}
6/6 [==============================] - 0s 2ms/step - loss: 0.0450 - mean_squared_error: 0.0450
Loss: 0.04498277232050896 Tiempo transcurrido: 30.976457595825195

--- Starting trial: run-42
{'layers': 3, 'num_units': 50, 'learning_rate': 0.001, 'batch_size': 1}
6/6 [==============================] - 0s 2ms/step - loss: 0.0425 - mean_squared_error: 0.0425
Loss: 0.04251638799905777 Tiempo transcurrido: 70.25398182868958

--- Starting trial: run-43
{'layers': 3, 'num_units': 5

6/6 [==============================] - 0s 2ms/step - loss: 0.0417 - mean_squared_error: 0.0417
Loss: 0.041682519018650055 Tiempo transcurrido: 17.91736602783203

--- Starting trial: run-60
{'layers': 3, 'num_units': 150, 'learning_rate': 0.001, 'batch_size': 1}
6/6 [==============================] - 0s 2ms/step - loss: 0.0766 - mean_squared_error: 0.0766
Loss: 0.0765555202960968 Tiempo transcurrido: 181.1593267917633

--- Starting trial: run-61
{'layers': 3, 'num_units': 150, 'learning_rate': 0.001, 'batch_size': 4}
6/6 [==============================] - 0s 2ms/step - loss: 0.0747 - mean_squared_error: 0.0747
Loss: 0.07471899688243866 Tiempo transcurrido: 25.57457399368286

--- Starting trial: run-62
{'layers': 3, 'num_units': 150, 'learning_rate': 0.001, 'batch_size': 8}
6/6 [==============================] - 0s 1ms/step - loss: 0.0519 - mean_squared_error: 0.0519
Loss: 0.05194811895489693 Tiempo transcurrido: 10.440757513046265

--- Starting trial: run-63
{'layers': 3, 'num_units': 2

6/6 [==============================] - 0s 2ms/step - loss: 0.0774 - mean_squared_error: 0.0774
Loss: 0.07740601152181625 Tiempo transcurrido: 14.569139242172241

--- Starting trial: run-80
{'layers': 4, 'num_units': 50, 'learning_rate': 0.001, 'batch_size': 8}
6/6 [==============================] - 0s 2ms/step - loss: 0.0473 - mean_squared_error: 0.0473
Loss: 0.04727548733353615 Tiempo transcurrido: 10.939963102340698

--- Starting trial: run-81
{'layers': 4, 'num_units': 100, 'learning_rate': 1e-05, 'batch_size': 1}
6/6 [==============================] - 0s 2ms/step - loss: 0.0513 - mean_squared_error: 0.0513
Loss: 0.05128227546811104 Tiempo transcurrido: 127.8565878868103

--- Starting trial: run-82
{'layers': 4, 'num_units': 100, 'learning_rate': 1e-05, 'batch_size': 4}
6/6 [==============================] - 0s 2ms/step - loss: 0.0497 - mean_squared_error: 0.0497
Loss: 0.04966979846358299 Tiempo transcurrido: 79.09989714622498

--- Starting trial: run-83
{'layers': 4, 'num_units': 1

6/6 [==============================] - 0s 1ms/step - loss: 0.0464 - mean_squared_error: 0.0464
Loss: 0.04640369489789009 Tiempo transcurrido: 111.97455835342407

--- Starting trial: run-100
{'layers': 4, 'num_units': 200, 'learning_rate': 1e-05, 'batch_size': 4}
6/6 [==============================] - 0s 2ms/step - loss: 0.0505 - mean_squared_error: 0.0505
Loss: 0.05048169940710068 Tiempo transcurrido: 53.87628936767578

--- Starting trial: run-101
{'layers': 4, 'num_units': 200, 'learning_rate': 1e-05, 'batch_size': 8}
6/6 [==============================] - 0s 1ms/step - loss: 0.0576 - mean_squared_error: 0.0576
Loss: 0.05764072388410568 Tiempo transcurrido: 37.73611807823181

--- Starting trial: run-102
{'layers': 4, 'num_units': 200, 'learning_rate': 0.0001, 'batch_size': 1}
6/6 [==============================] - 0s 1ms/step - loss: 0.0481 - mean_squared_error: 0.0481
Loss: 0.04808483645319939 Tiempo transcurrido: 76.91801047325134

--- Starting trial: run-103
{'layers': 4, 'num_unit

In [20]:
filename = "historial_jla_tunning.txt"
df = pd.DataFrame(datos, columns = ["Deep size", "Num units", "Learning rate", "Batch size", "MSE", "Tiempo de ejecución"])

df.sort_values(by=["MSE", "Tiempo de ejecución"], ascending=[0,1], ignore_index=True, inplace=True)

df.to_csv(filename, header=True, index=False, sep='\t', mode='w') # a=append, w=overwrite

In [21]:
total_time = float(np.sum(df[["Tiempo de ejecución"]])/60)

print("Elapsed time {:.3f} minutes".format(total_time))

Elapsed time 20.053 minutes


In [23]:
df["Tiempo de ejecución"]

0     16.857623
1     14.317844
2     68.457287
3     19.127944
4     67.325694
5      9.881540
6     76.629540
7     21.154999
8     58.180007
9     61.509559
10    61.211818
11    12.584826
12    53.191603
13    82.672153
14    37.736118
15    32.484522
16    21.397434
17    82.638258
18    13.525098
19    53.640059
20    10.440758
21    41.728017
22    41.899422
23    10.939963
24    19.949551
25    19.798563
26    13.904052
27    19.045640
28    30.976458
29    24.814155
30     8.827588
31    22.065886
32    17.917366
33    15.417565
34    22.201454
35    18.747259
Name: Tiempo de ejecución, dtype: float64